In [1]:
#importing all libraries (made my own library for functions I'm repeatedly using in other notebooks)
import numpy as np
import pandas as pd
import sqlite3
import bq_helper
from bq_helper import BigQueryHelper
from datetime import datetime
from lib_DMH import gsodqueryyear
from lib_DMH import mapplot
import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import shapely.geometry as sgeom
import shapely
import matplotlib.pyplot as plt
import time

In [2]:
#Calling the FIA BigQuery API
usfs = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="usfs_fia")

In [3]:
#Defining the states in the southern region. We could expand it to include other states
states =('Alabama', 'Arkansas', 'Florida', 
         'Georgia', 'Kentucky', 'Louisiana', 
         'Mississippi', 'North Carolina', 
         'Oklahoma', 'South Carolina', 'Tennessee', 
         'Texas', 'Virginia')

In [4]:
#Defining the top 10 tree species in the US. We can adjust it to other specific trees that we want to explore.
trees = (316,131,611,202,746,318,12,491,108,802)

#Defining the query for what data I want from the FIA API
query1 = f"""
        SELECT
            plot_phase_2_plot_number,
            plot_state_code,
            plot_county_code,
            measurement_year,
            measurement_month,
            latitude,
            longitude,
            species_code,
            AVG(total_height) as avg_height,
            AVG(current_diameter) as avg_diameter,
            SUM(total_height) as total_height,
        FROM
            `bigquery-public-data.usfs_fia.plot_tree`
        WHERE
             plot_state_code_name in {states} AND species_code in {trees}
        GROUP BY
             plot_phase_2_plot_number,
             plot_state_code,
             plot_county_code,
             measurement_year,
             measurement_month,
             latitude,
             longitude,
             species_code
        ;
                """

In [5]:
#Creating a pandas dataframe for the data requested
plots = usfs.query_to_pandas_safe(query1, max_gb_scanned=10)
#plots = plots.dropna()
#plots['date'] = pd.to_datetime(plots[['measurement_year','measurement_month']].rename(columns = {'measurement_year':'year','measurement_month':'month'}).assign(day=1).astype(int))
plots = plots.reset_index(drop=True)

In [6]:
#Adding a column that takes the state and county codes to make a uniquely identying FIPS code 
plots_c_s = plots.copy()
plots_c_s[['plot_state_code','plot_county_code']] = plots_c_s[['plot_state_code','plot_county_code']].astype(str)
plots_c_s = plots_c_s.assign(fips_code = lambda x: x.plot_state_code + '_' + x.plot_county_code)

#Reducing the size of the dataframe to just include data past the year 2000
plots_c_s = plots_c_s.loc[(plots_c_s.measurement_year >= 2000)]
plots_c_s

,plot_phase_2_plot_number,plot_state_code,plot_county_code,measurement_year,measurement_month,latitude,longitude,species_code,avg_height,avg_diameter,total_height,fips_code
0,51,5,125,2019.0,1.0,34.517532,-92.267296,131,49.704545,8.550000,2187.0,5_125
1,4,45,35,2019.0,1.0,32.956730,-80.323410,131,79.300000,10.226667,2379.0,45_35
2,15,28,85,2019.0,3.0,31.453297,-90.610802,131,NaN,NaN,NaN,28_85
3,39,28,23,2019.0,3.0,32.109669,-88.490593,131,67.657895,8.870526,2571.0,28_23
15,65,1,51,2019.0,3.0,32.644661,-86.041603,131,87.000000,12.643750,696.0,1_51
...,...,...,...,...,...,...,...,...,...,...,...,...
521793,114,48,419,2018.0,5.0,31.737793,-93.966888,131,118.000000,25.200001,118.0,48_419
521794,48,1,99,2018.0,1.0,31.563969,-87.338821,131,63.000000,10.090000,63.0,1_99
521795,40,48,403,2018.0,6.0,31.328863,-93.719330,802,62.000000,9.300000,62.0,48_403
521796,54,1,57,2018.0,10.0,33.678440,-87.451668,131,29.000000,5.000000,29.0,1_57


In [7]:
#Calling the NOAA GSOD BigQuery API 
noaa_gsod = bq_helper.BigQueryHelper(active_project= "bigquery-public-data", 
                                     dataset_name= "noaa_gsod")

In [8]:
#defining the query to grab the station table
query1 = """
            SELECT 
                usaf AS Station_number, 
                lat AS Latitude, 
                lon AS Longitude, 
            FROM 
                `bigquery-public-data.noaa_gsod.stations` 
            WHERE 
                country = 'US' AND lat IS NOT NULL AND lon IS NOT NULL AND NOT (lat = 0.0 AND lon = 0.0) AND NOT usaf = '999999' 
        """

In [9]:
#Creating a dataframe from the station numbers. Some of the stations are in the same location (lat and long).
#Dropped these excess stations
stations1 = noaa_gsod.query_to_pandas_safe(query1, max_gb_scanned=10)
stations = stations1.copy()
stations = stations.drop(stations.loc[stations.Station_number.duplicated(keep='last')].index)

In [10]:
#Defined a function that matches the stations in the gsod data with latitude and longitude
def latlong(gsod,stations):
    
    lat = np.empty(len(gsod.Station_number))
    long = np.empty(len(gsod.Station_number))
    gsod_copy = gsod.copy()
    for i in range(len(gsod.Station_number)):
        lat[i] = np.array(stations.loc[stations.Station_number == gsod.Station_number[i]].Latitude)[0]
        long[i] = np.array(stations.loc[stations.Station_number == gsod.Station_number[i]].Longitude)[0]
    gsod_copy['Latitude'] = lat
    gsod_copy['Longitude'] = long
    return gsod_copy

#Defined a function that links the plot locations in the data with the nearest GSOD station
def find_nearest3(lat,long,df):
    
    index_nearest = np.sqrt((lat-df.Latitude)**2 + (long-df.Longitude)**2).idxmin()
    return df.Station_number[index_nearest]

#Defined a function that applies the find_nearest3 to every plot in the plot dataframe
def get_station3(sta, plt):
    
    plt_copy = plt.copy()
    nstation = np.empty(len(plt_copy)).astype(str)
    nstation_ind = np.empty(len(plt_copy))
    
    for i in range(len(plt_copy)):
        
        nstation[i] = find_nearest3(round(plt_copy.latitude[i],3),round(plt_copy.longitude[i],3),sta)
        
    plt_copy['nearest_station'] = nstation
    
    return plt_copy

#Defined a function that adds the temperature, dewpoint, and pressure at the station nearest each plot
def add_temp(df,gsod):
    feats = np.empty((len(df),3))
    for i in range(len(df)):
        features = np.array(gsod.loc[(gsod.Station_number == df.nearest_station[i])][['Mean_temp','Mean_dwp','Mean_prcp']])
        
        #if there is no feature data available at the station, it will put in place a NaN 
        try:
            feats[i][0] =features[0][0]
        except:
            feats[i][0] = np.nan
        try:
            feats[i][1] =features[0][1]
        except:
            feats[i][1] = np.nan
        try:
            feats[i][2] =features[0][2]
        except:
            feats[i][2] = np.nan
    df = df.assign(mean_temp = feats[:,0], mean_dwp = feats[:,1], mean_prcp = feats[:,2])
    return df


In [11]:
#Creating a dataframe to store the plot data with added GSOD station features
plot_stations = pd.DataFrame()
stations_tuple = tuple(stations.Station_number)

In [12]:
#putting it all together
t0 = time.perf_counter()
for i in range(2019-2000+1):
    
    year = 2000+i
    plot_year = plots_c_s.loc[plots_c_s.measurement_year == year].reset_index(drop=True)
    gsod = gsodqueryyear(year, stations_tuple, noaa_gsod)
    gsod.Year = gsod.Year.astype(float)
    #gsod['date'] = pd.to_datetime(gsod[['Year','Month']].assign(day=1))
    gsod = latlong(gsod,stations)
    plot_station = get_station3(gsod, plot_year)
    plot_station = add_temp(plot_station,gsod)
    plot_stations = pd.concat([plot_stations,plot_station])

t1 = time.perf_counter()
total_t = t1-t0

In [13]:
total_t

493.83679196400044

In [14]:
#Averaging the data over each county each year
county_tree_df = plot_stations.groupby(['measurement_year','fips_code','species_code']).mean().reset_index().drop(['measurement_month','plot_phase_2_plot_number'],axis=1)
county_tree_df

,measurement_year,fips_code,species_code,latitude,longitude,avg_height,avg_diameter,total_height,mean_temp,mean_dwp,mean_prcp
0,2000.0,12_129,316,30.102601,-84.371906,42.750000,8.775000,70.0,67.335165,56.667582,0.121016
1,2000.0,12_129,491,30.181875,-84.324379,43.700000,6.360000,140.5,67.335165,56.667582,0.121016
2,2000.0,12_129,611,30.236635,-84.251587,71.181818,10.979091,783.0,67.335165,56.667582,0.121016
3,2000.0,13_1,131,31.817596,-82.147518,45.734028,7.584750,522.5,67.600990,55.544930,0.112880
4,2000.0,13_1,316,31.781965,-82.212811,42.882715,5.720597,379.8,67.583865,55.598961,0.110349
...,...,...,...,...,...,...,...,...,...,...,...
59228,2019.0,5_87,802,35.798729,-93.771492,85.500000,17.190000,204.0,57.774725,49.717582,0.182088
59229,2019.0,5_91,131,33.153614,-93.967422,40.804348,6.864783,641.5,64.118630,54.040274,0.143342
59230,2019.0,5_91,316,33.026779,-93.994385,57.000000,5.400000,57.0,64.118630,54.040274,0.143342
59231,2019.0,5_91,611,33.153614,-93.967422,37.791667,4.933333,219.0,64.118630,54.040274,0.143342


In [15]:
#saving the results
county_tree_df.to_csv('county_tree_temp_dwp_prcp.csv')